G2M insight for Cab Investment firm model building

In [3]:
#G2M insight for Cab Investment firm 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_curve
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


In [4]:
cab_data = pd.read_csv("Desktop/Cab_Data.csv")
CustomerID_data=pd.read_csv("Desktop/Customer_ID.csv")
TransactionID_data=pd.read_csv("Desktop/Transaction_ID.csv")
City_data=pd.read_csv("Desktop/City.csv")

cab_data.head()



,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
0,10000011,08-01-2016,Pink Cab,ATLANTA GA,30.45,370.95,313.635
1,10000012,06-01-2016,Pink Cab,ATLANTA GA,28.62,358.52,334.854
2,10000013,02-01-2016,Pink Cab,ATLANTA GA,9.04,125.20,97.632
3,10000014,07-01-2016,Pink Cab,ATLANTA GA,33.17,377.40,351.602
4,10000015,03-01-2016,Pink Cab,ATLANTA GA,8.73,114.62,97.776


In [5]:
cab_data.tail()


,Transaction ID,Date of Travel,Company,City,KM Travelled,Price Charged,Cost of Trip
359387,10440101,08-01-2018,Yellow Cab,WASHINGTON DC,4.80,69.24,63.3600
359388,10440104,04-01-2018,Yellow Cab,WASHINGTON DC,8.40,113.75,106.8480
359389,10440105,05-01-2018,Yellow Cab,WASHINGTON DC,27.75,437.07,349.6500
359390,10440106,05-01-2018,Yellow Cab,WASHINGTON DC,8.80,146.19,114.0480
359391,10440107,02-01-2018,Yellow Cab,WASHINGTON DC,12.76,191.58,177.6192


In [6]:
TransactionID_data.head()

,Transaction ID,Customer ID,Payment_Mode
0,10000011,29290,Card
1,10000012,27703,Card
2,10000013,28712,Cash
3,10000014,28020,Cash
4,10000015,27182,Card


In [7]:
TransactionID_data.tail()

,Transaction ID,Customer ID,Payment_Mode
440093,10440104,53286,Cash
440094,10440105,52265,Cash
440095,10440106,52175,Card
440096,10440107,52917,Card
440097,10440108,51587,Card


In [8]:
CustomerID_data.head()

,Customer ID,Gender,Age,Income (USD/Month)
0,29290,Male,28,10813
1,27703,Male,27,9237
2,28712,Male,53,11242
3,28020,Male,23,23327
4,27182,Male,33,8536


In [9]:
CustomerID_data.tail()

,Customer ID,Gender,Age,Income (USD/Month)
49166,12490,Male,33,18713
49167,14971,Male,30,15346
49168,41414,Male,38,3960
49169,41677,Male,23,19454
49170,39761,Female,32,10128


In [10]:
City_data.head()

,City,Population,Users
0,NEW YORK NY,"8,405,837","302,149"
1,CHICAGO IL,"1,955,130","164,468"
2,LOS ANGELES CA,"1,595,037","144,132"
3,MIAMI FL,"1,339,155","17,675"
4,SILICON VALLEY,"1,177,609","27,247"


In [11]:
City_data.tail()

,City,Population,Users
15,SACRAMENTO CA,"545,776","7,044"
16,PITTSBURGH PA,"542,085","3,643"
17,WASHINGTON DC,"418,859","127,001"
18,NASHVILLE TN,"327,225","9,270"
19,BOSTON MA,"248,968","80,021"


In [12]:
# Check the shape of whole dataset
print("shape of Cab_Data", cab_data.shape)
print("shape of Customer_ID", CustomerID_data.shape)
print("shape of Transaction_ID", TransactionID_data.shape)
print("shape of City Data", City_data.shape)


shape of Cab_Data (359392, 7)
shape of Customer_ID (49171, 4)
shape of Transaction_ID (440098, 3)
shape of City Data (20, 3)


In [13]:
# check the basic information 
print(cab_data.info())
print(CustomerID_data.info())
print(TransactionID_data.info())
print(City_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359392 entries, 0 to 359391
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Transaction ID  359392 non-null  int64  
 1   Date of Travel  359392 non-null  object 
 2   Company         359392 non-null  object 
 3   City            359392 non-null  object 
 4   KM Travelled    359392 non-null  float64
 5   Price Charged   359392 non-null  float64
 6   Cost of Trip    359392 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 19.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49171 entries, 0 to 49170
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Customer ID         49171 non-null  int64 
 1   Gender              49171 non-null  object
 2   Age                 49171 non-null  int64 
 3   Income (USD/Month)  49171 non-null  int64 
dtypes: int6

In [14]:
cab_data.isnull().sum() + cab_data.isna().sum()

Transaction ID    0
Date of Travel    0
Company           0
City              0
KM Travelled      0
Price Charged     0
Cost of Trip      0
dtype: int64

In [15]:
TransactionID_data.isnull().sum() + TransactionID_data.isna().sum()

Transaction ID    0
Customer ID       0
Payment_Mode      0
dtype: int64

In [16]:
CustomerID_data.isnull().sum() + CustomerID_data.isna().sum()

Customer ID           0
Gender                0
Age                   0
Income (USD/Month)    0
dtype: int64

In [17]:
City_data.isnull().sum() + City_data.isna().sum()

City          0
Population    0
Users         0
dtype: int64

In [18]:
for col in cab_data.columns:
    if ' ' in col:
        cab_data = cab_data.rename(columns={col:col.replace(' ','_')})
for col in TransactionID_data.columns:
    if ' ' in col:
        TransactionID_data = TransactionID_data.rename(columns={col:col.replace(' ','_')})
for col in CustomerID_data.columns:
    if ' ' in col:
        CustomerID_data = CustomerID_data.rename(columns={col:col.replace(' ','_')})
cab_data['Date_of_Travel'] = pd.to_datetime(cab_data['Date_of_Travel'])

months=[]
years =[]
for i in range(len(cab_data['Date_of_Travel'])):
    months.append(cab_data['Date_of_Travel'][i].month)
    years.append(cab_data['Date_of_Travel'][i].year)
cab_data['Month'] = months
cab_data['Year'] = years

cab_data.drop('Date_of_Travel', axis=1, inplace=True)


In [19]:
PinkCab = cab_data[cab_data['Company']=='Pink Cab']
YellowCab = cab_data[cab_data['Company']=='Yellow Cab']
cab_data.head()

#Creating Master Data
master_data = cab_data.set_index([cab_data.index,'Transaction_ID']).join(TransactionID_data.set_index(['Transaction_ID']))

master_data = master_data.set_index([master_data.index,'Customer_ID']).join(CustomerID_data.set_index(['Customer_ID']))

master_data = master_data.set_index([master_data.index,'City']).join(City_data.set_index(['City']))

master_data.head()


,,,,Company,KM_Travelled,Price_Charged,Cost_of_Trip,Month,Year,Payment_Mode,Gender,Age,Income_(USD/Month),Population,Users
,Transaction_ID,Customer_ID,City,,,,,,,,,,,,
0,10000011,29290,ATLANTA GA,Pink Cab,30.45,370.95,313.635,8,2016,Card,Male,28,10813,"814,885","24,701"
1,10000012,27703,ATLANTA GA,Pink Cab,28.62,358.52,334.854,6,2016,Card,Male,27,9237,"814,885","24,701"
2,10000013,28712,ATLANTA GA,Pink Cab,9.04,125.20,97.632,2,2016,Cash,Male,53,11242,"814,885","24,701"
3,10000014,28020,ATLANTA GA,Pink Cab,33.17,377.40,351.602,7,2016,Cash,Male,23,23327,"814,885","24,701"
4,10000015,27182,ATLANTA GA,Pink Cab,8.73,114.62,97.776,3,2016,Card,Male,33,8536,"814,885","24,701"


In [20]:
master_data.tail()

,,,,Company,KM_Travelled,Price_Charged,Cost_of_Trip,Month,Year,Payment_Mode,Gender,Age,Income_(USD/Month),Population,Users
,Transaction_ID,Customer_ID,City,,,,,,,,,,,,
359387,10440101,52392,WASHINGTON DC,Yellow Cab,4.80,69.24,63.3600,8,2018,Cash,Male,24,15651,"418,859","127,001"
359388,10440104,53286,WASHINGTON DC,Yellow Cab,8.40,113.75,106.8480,4,2018,Cash,Male,32,6528,"418,859","127,001"
359389,10440105,52265,WASHINGTON DC,Yellow Cab,27.75,437.07,349.6500,5,2018,Cash,Male,56,7966,"418,859","127,001"
359390,10440106,52175,WASHINGTON DC,Yellow Cab,8.80,146.19,114.0480,5,2018,Card,Male,32,6423,"418,859","127,001"
359391,10440107,52917,WASHINGTON DC,Yellow Cab,12.76,191.58,177.6192,2,2018,Card,Male,20,11284,"418,859","127,001"


In [21]:
master_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 359392 entries, (0, 10000011, 29290, 'ATLANTA GA') to (359391, 10440107, 52917, 'WASHINGTON DC')
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Company             359392 non-null  object 
 1   KM_Travelled        359392 non-null  float64
 2   Price_Charged       359392 non-null  float64
 3   Cost_of_Trip        359392 non-null  float64
 4   Month               359392 non-null  int64  
 5   Year                359392 non-null  int64  
 6   Payment_Mode        359392 non-null  object 
 7   Gender              359392 non-null  object 
 8   Age                 359392 non-null  int64  
 9   Income_(USD/Month)  359392 non-null  int64  
 10  Population          359392 non-null  object 
 11  Users               359392 non-null  object 
dtypes: float64(3), int64(4), object(5)
memory usage: 60.3+ MB


In [22]:
YellowCab = master_data[master_data['Company']=='Yellow Cab']
PinkCab = master_data[master_data['Company']=='Pink Cab']
YellowCab.drop(['Company', 'Payment_Mode', 'Gender','Population','Users'], axis=1, inplace=True)
PinkCab.drop(['Company', 'Payment_Mode', 'Gender','Population','Users'], axis=1, inplace=True)
YellowCab_X = YellowCab.drop(['Price_Charged'], axis=1)
YellowCab_y = YellowCab['Price_Charged'].copy()
PinkCab_X = PinkCab.drop(['Price_Charged'], axis=1)
PinkCab_y = PinkCab['Price_Charged'].copy()


C:\Users\iTs\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [24]:
X_train, X_test, y_train, y_test = train_test_split(YellowCab_X, YellowCab_y,test_size=0.2,random_state=42) 

# Simple Linear Regression
model_simple_linear = LinearRegression(fit_intercept=True) # data is not centered, we need an intercept!
model_simple_linear.fit(X_train, y_train)
LinearRegression(fit_intercept=True)
preds_valid = model_simple_linear.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, preds_valid)) 
print(f'Train score : {rmse}')

predicted = model_simple_linear.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
print(f'Test score : {rmse}')


Train score : 146.3384430110292
Test score : 146.75321496823184


In [25]:
model = LinearRegression(fit_intercept=True) 
model.fit(X_train, y_train)
LinearRegression(fit_intercept=True) 
accuracyl_YellowCab = model.score(X_test,y_test)
print('The accuracy is: ', accuracyl_YellowCab)



The accuracy is:  0.7415043837937295


In [26]:
dtree = DecisionTreeRegressor()

dtree.fit(X_train, y_train)

preds_valid = dtree.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, preds_valid)) 
print(f'Train score : {rmse}')
predicted = dtree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
print(f'Test score : {rmse}')


Train score : 0.0885890600551407
Test score : 210.30843292007225


In [27]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
DecisionTreeRegressor()
accuracyd_YellowCab = model.score(X_test,y_test)
print('The accuracy is: ', accuracyd_YellowCab)



The accuracy is:  0.4680797456460495


In [28]:
#Random Forest

#Yellow Cab
rf = RandomForestRegressor(n_estimators = 100, random_state = 0)
rf.fit(X_train, y_train)
 
preds_valid = rf.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, preds_valid))
 
print(f'Train score : {rmse}')
predicted = rf.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
print(f'Test score : {rmse}')


Train score : 56.05418447867246
Test score : 150.54193106858702


In [29]:
model = RandomForestRegressor(n_estimators = 100, random_state = 0) 
model.fit(X_train, y_train)
RandomForestRegressor(n_estimators = 100, random_state = 0)
accuracyrf_YellowCab = model.score(X_test,y_test)
print('The accuracy is: ', accuracyrf_YellowCab)



The accuracy is:  0.7279849714001346


In [30]:
#Yellow Cab
gb_model = XGBRegressor(n_estimators = 1000, learning_rate = 0.05) 
gb_model.fit(X_train, y_train)
 
preds_valid = gb_model.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, preds_valid)) 
print(f'Train score : {rmse}')
predicted = gb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
print(f'Test score : {rmse}')


Train score : 128.1342040274574
Test score : 134.9287638326252


In [31]:
model = XGBRegressor(n_estimators = 1000, learning_rate = 0.05)
model.fit(X_train, y_train)
XGBRegressor(n_estimators = 1000, learning_rate = 0.05)
accuracygb_YellowCab = model.score(X_test,y_test)
print('The accuracy is: ', accuracygb_YellowCab)


The accuracy is:  0.7814821006474011


In [32]:
#Yellow Cab
lgbm_model = LGBMRegressor(metric = 'rmse', n_jobs = -1, n_estimators = 5000, learning_rate = 0.01)
lgbm_model.fit(X_train,  y_train, 
               eval_set = [(X_test,  y_test)],
               early_stopping_rounds = 40,
               verbose = False)

preds_valid = lgbm_model.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, preds_valid)) 
print(f'Train score : {rmse}')
predicted = lgbm_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
print(f'Test score : {rmse}')


C:\Users\iTs\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\iTs\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Train score : 133.04133225355477
Test score : 139.53695721439757


In [33]:
model = LGBMRegressor(metric = 'rmse', n_jobs = -1, n_estimators = 5000, learning_rate = 0.01)
model.fit(X_train, y_train)
LGBMRegressor(metric = 'rmse', n_jobs = -1, n_estimators = 5000, learning_rate = 0.01)
accuracylgbm_YellowCab = model.score(X_test,y_test)
print('The accuracy is: ',  accuracylgbm_YellowCab)


The accuracy is:  0.7663012278460316


In [35]:
#Pink Cab
X_train, X_test, y_train, y_test = train_test_split(PinkCab_X, PinkCab_y, test_size=0.2, random_state=42) 

model_simple_linear = LinearRegression(fit_intercept=True) # data is not centered, we need an intercept!
model_simple_linear.fit(X_train, y_train)
preds_valid = model_simple_linear.predict(X_train)
rmse = np.sqrt(mean_squared_error(y_train, preds_valid))
print(f'Train score : {rmse}')
predicted = model_simple_linear.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
print(f'Test score : {rmse}')


Train score : 67.38846921384408
Test score : 67.98726328989346


In [36]:
model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
LinearRegression()
accuracyl_PinkCab = model.score(X_test,y_test)
print('The accuracy is: ', accuracyl_PinkCab)


The accuracy is:  0.8600088200328847


In [37]:
dtree = DecisionTreeRegressor()

dtree.fit(X_train, y_train)

preds_valid = dtree.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, preds_valid)) 
print(f'Train score : {rmse}')
predicted = dtree.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
print(f'Test score : {rmse}')


Train score : 0.0
Test score : 98.78242706324811


In [38]:
model = DecisionTreeRegressor()
model.fit(X_train, y_train)
DecisionTreeRegressor()
accuracyd_PinkCab = model.score(X_test,y_test)
print('The accuracy is: ' , accuracyd_PinkCab)


The accuracy is:  0.7057234792674434


In [39]:
#Random Forest

rf = RandomForestRegressor(n_estimators = 100, random_state = 0)
rf.fit(X_train, y_train)
 
preds_valid = rf.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, preds_valid)) 
print(f'Train score : {rmse}')
predicted = rf.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
print(f'Test score : {rmse}')


Train score : 25.858508932989594
Test score : 69.71772672156538


In [40]:
model = RandomForestRegressor(n_estimators = 100, random_state = 0) 
model.fit(X_train, y_train)
RandomForestRegressor(n_estimators = 100, random_state = 0)
accuracyrf_PinkCab = model.score(X_test,y_test)
print('The accuracy is: ' , accuracyrf_PinkCab)



The accuracy is:  0.8527918041862741


In [41]:
#Pink Cab
gb_model = XGBRegressor(n_estimators = 1000, learning_rate = 0.05) 
gb_model.fit(X_train, y_train)
 
preds_valid = gb_model.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, preds_valid)) 
print(f'Train score : {rmse}')
predicted = gb_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
print(f'Test score : {rmse}')


Train score : 56.257579458387525
Test score : 68.05501636022264


In [42]:
model = XGBRegressor(n_estimators = 1000, learning_rate = 0.05)
model.fit(X_train, y_train)
XGBRegressor(n_estimators = 1000, learning_rate = 0.05)
accuracygb_PinkCab = model.score(X_test,y_test)
print('The accuracy is: ',  accuracygb_PinkCab)


The accuracy is:  0.8597296630884013


In [43]:
lgbm_model = LGBMRegressor(metric = 'rmse', n_jobs = -1, n_estimators = 5000, learning_rate = 0.01)
lgbm_model.fit(X_train,  y_train, 
               eval_set = [(X_test,  y_test)],
               early_stopping_rounds = 40,
               verbose = False)

preds_valid = lgbm_model.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, preds_valid)) 
print(f'Train score : {rmse}')

predicted = lgbm_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
print(f'Test score : {rmse}')



C:\Users\iTs\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\iTs\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Train score : 65.33921341293481
Test score : 67.21044264668784


In [44]:
model = LGBMRegressor(metric = 'rmse', n_jobs = -1, n_estimators = 5000, learning_rate = 0.01)
model.fit(X_train, y_train)
LGBMRegressor(metric = 'rmse', n_jobs = -1, n_estimators = 5000, learning_rate = 0.01)
accuracylgbm_PinkCab = model.score(X_test,y_test)
print('The accuracy is: ' , accuracylgbm_PinkCab)



The accuracy is:  0.8595862480923745


In [45]:
results = pd.DataFrame([str(accuracyl_YellowCab *100) + '%', str(accuracyd_YellowCab *100) + '%',str(accuracyrf_YellowCab *100) + '%',str(accuracygb_YellowCab *100) + '%', str(accuracylgbm_YellowCab *100) + '%'],
                            index=['Linear Regression',
                                  'Decision Tree', 'Random Forest','Gradient Boosting', 'Light Gradient Boosting'],columns=['Accuracy'])
results




,Accuracy
Linear Regression,74.15043837937296%
Decision Tree,46.80797456460495%
Random Forest,72.79849714001347%
Gradient Boosting,78.14821006474011%
Light Gradient Boosting,76.63012278460316%


In [46]:
results = pd.DataFrame([str(accuracyl_PinkCab *100) + '%', str(accuracyd_PinkCab *100) + '%',str(accuracyrf_PinkCab *100) + '%',str(accuracygb_PinkCab *100) + '%', str(accuracylgbm_PinkCab *100) + '%'],
             
                             index=['Linear Regression',
                                  'Decision Tree', 'Random Forest', 'Gradient Boosting', 'Light Gradient Boosting'],columns=['Accuracy'])
results

,Accuracy
Linear Regression,86.00088200328847%
Decision Tree,70.57234792674434%
Random Forest,85.2791804186274%
Gradient Boosting,85.97296630884013%
Light Gradient Boosting,85.95862480923745%
